In [1]:
import pandas as pd

In [12]:
data = pd.read_csv("cnvator_data_archaic_overlapped.csv")
anotation = pd.read_csv('/branchinecta/jbazanwilliamson/SGDP_anotation.csv', sep=',', encoding='latin-1')#,  index_col=1)

In [13]:
cnv = data.pivot_table(index=["Chr", "Start", "End"], 
                    columns='Sample_ID', 
                    values='RD').reset_index()


cnv

Sample_ID,Chr,Start,End,LP6005441-DNA_A01,LP6005441-DNA_A03,LP6005441-DNA_A04,LP6005441-DNA_A05,LP6005441-DNA_A06,LP6005441-DNA_A08,LP6005441-DNA_A09,...,SS6004471,SS6004472,SS6004473,SS6004474,SS6004475,SS6004476,SS6004477,SS6004478,SS6004479,SS6004480
0,1,228416,267160,2.089230,1.098060,1.088959,1.94512,1.743018,1.263408,1.821840,...,1.99166,0.782568,1.862735,0.739641,1.551208,2.087340,0.971431,1.032574,1.820703,1.947480
1,1,521526,568203,0.351718,0.381499,0.353139,NaN,1.501635,0.449098,0.314463,...,1.22385,0.189764,0.289165,0.355864,0.716763,1.273283,0.384140,0.404601,0.918972,0.366604
2,1,563872,572534,NaN,NaN,NaN,NaN,NaN,0.609951,0.505381,...,NaN,NaN,NaN,NaN,NaN,NaN,0.478341,0.700392,NaN,NaN
3,1,899742,901754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,974768,978161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.338520,0.332834,0.222146,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,22,50342488,50345958,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.269240,NaN,1.386350
4611,22,50443466,50446957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4612,22,50649161,50654860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4613,22,50754570,50756386,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.305850,NaN,NaN


In [14]:
coordinates = cnv.iloc[:,0:3]
cnvs = cnv.iloc[:,4:284]
cnvs


Sample_ID,LP6005441-DNA_A03,LP6005441-DNA_A04,LP6005441-DNA_A05,LP6005441-DNA_A06,LP6005441-DNA_A08,LP6005441-DNA_A09,LP6005441-DNA_A10,LP6005441-DNA_A11,LP6005441-DNA_A12,LP6005441-DNA_B01,...,SS6004471,SS6004472,SS6004473,SS6004474,SS6004475,SS6004476,SS6004477,SS6004478,SS6004479,SS6004480
0,1.098060,1.088959,1.94512,1.743018,1.263408,1.821840,2.258200,1.590700,0.793560,1.68227,...,1.99166,0.782568,1.862735,0.739641,1.551208,2.087340,0.971431,1.032574,1.820703,1.947480
1,0.381499,0.353139,NaN,1.501635,0.449098,0.314463,0.152128,1.525555,5.998952,NaN,...,1.22385,0.189764,0.289165,0.355864,0.716763,1.273283,0.384140,0.404601,0.918972,0.366604
2,NaN,NaN,NaN,NaN,0.609951,0.505381,0.611279,1.553540,16.630300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.478341,0.700392,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,0.676387,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.338520,0.332834,0.222146,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.269240,NaN,1.386350
4611,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.305850,NaN,NaN


In [16]:
anotation.groupby('10-Region')

In [21]:
regions = anotation['10-Region'].unique()
for i in regions:
    anotation.groupby('10-Region')[str(i)]

KeyError: 'Column not found: WestEurasia'

In [22]:
gk = anotation.groupby('10-Region')

In [23]:
gk.first()

,2-ena_id,3-Illumina_ID,4-reich_id,5-ena_ftp_bam_ptr,6-Sequencing_Panel,7-Gender,8-Population_ID,9-Contributor,11-Country,12-Town,13-Latitude,14-Longitude,15-DNA_Source,16-BioSamples Accession,17-Embargo
10-Region,,,,,,,,,,,,,,,
Africa,SAMEA3302647,LP6005442-DNA_G11,S_Mende-1,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324275/LP6005...,C,M,Mende,1KG,SierraLeone,?,8.500000,-13.2,Genomic_from_cell_lines,SAMEA3302647,FullyPublic
America,SAMEA3302618,LP6005443-DNA_G11,S_Mixtec-1,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324341/LP6005...,C,M,Mixtec,William Klitz / Cheryl Winkler,Mexico,San Andres Nuxino,17.000000,-97.0,Genomic_from_cell_lines,SAMEA3302618,FullyPublic
CentralAsiaSiberia,SAMEA3302764,LP6005442-DNA_F12,S_Tubalar-1,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324267/LP6005...,C,F,Tubalar,Rem Sukernik / Stanislav Dryomov,Russia,"Artybash and Kebezen, Altai-Sayan Upland",51.133333,87.0,Genomic_from_blood,SAMEA3302764,FullyPublic
EastAsia,SAMEA3302615,LP6005442-DNA_G01,S_Yi-1,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324268/LP6005...,C,M,Yi,Howard Cann,China,?,28.000000,103.0,Genomic_from_cell_lines,SAMEA3302615,FullyPublic
Oceania,SAMEA3302905,LP6005592-DNA_B02,S_Maori-1,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324378/LP6005...,C,M,Maori,Coriell,New Zealand,Cell_line_repository_sampling_location_unknown,-41.300000,174.5,Genomic_from_cell_lines,SAMEA3302905,FullyPublic
SouthAsia,SAMEA3302616,LP6005441-DNA_D07,S_Makrani-2,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324870/LP6005...,C,F,Makrani,Howard Cann,Pakistan,?,26.000000,64.0,Genomic_from_cell_lines,SAMEA3302616,FullyPublic
WestEurasia,SAMEA3302859,LP6005443-DNA_B06,S_Icelandic-2,ftp.sra.ebi.ac.uk/vol1/ERZ324/ERZ324295/LP6005...,C,F,Icelandic,Coriell,Iceland,Cell_line_repository_sampling_location_unknown,64.100000,-21.9,Genomic_from_cell_lines,SAMEA3302859,FullyPublic
